需求：

北京明天的天气怎么样？上海呢？ （通过两次对话实现）

以ReAct模式为例进行实现：



In [2]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import os
import dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
dotenv.load_dotenv()

# 读取配置文件的信息
# os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
os.environ['TAVILY_API_KEY'] = "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"

# 获取Tavily搜索工具的实例
search = TavilySearchResults(max_results=3)

# 获取一个搜索的工具
search_tool = Tool(
    func=search.run,
    name="Search",
    description="用于检索互联网上的信息",
)


# 获取大语言模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
)

# 获取一个记忆的实例：ConversationBufferMemory
memory = ConversationBufferMemory(
    return_messages=True,
    memory_key="chat_history"  #此时必须设置memory_key,且值为chat_history。因为提示词模板中使用的是chat_history。二者必须一致！
)

# 获取Agent的实例
agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION

# 获取AgentExecutor的实例
agent_executor = initialize_agent(
    tools = [search_tool],
    llm = llm,
    agent = agent,
    verbose = True, # 显示详细的日志信息
    memory = memory,
)


# 通过AgentExecutor 调用invoke(),并得到响应
result = agent_executor.invoke("查询北京今天的天气情况")


# 处理响应数据
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: 北京今天的天气情况
```
Observation: [{'title': '中国气象局-天气预报- 北京', 'url': 'https://weather.cma.cn/web/weather/54511.html', 'content': '| 气压 | 1009.1hPa | 1007.6hPa | 1006.1hPa | 1005.9hPa | 1005.8hPa | 1005.5hPa | 1005.2hPa | 1005.9hPa |\n| 湿度 | 89.8% | 74.8% | 69.4% | 71.6% | 61% | 98.8% | 92.8% | 90.6% |\n| 云量 | 10.1% | 80% | 80% | 80% | 80% | 10.1% | 10.1% | 15% | [...] | 气压 | 1010.4hPa | 1010.1hPa | 1009.9hPa | 1011.2hPa | 1010.3hPa | 1008.9hPa | 1007.1hPa | 1009hPa |\n| 湿度 | 82.3% | 83.9% | 89.4% | 90.3% | 77.3% | 73.1% | 70.4% | 63.8% |\n| 云量 | 70% | 70% | 70% | 70% | 70% | 70% | 76.7% | 70% | [...] | 气压 | 1011.2hPa | 1010.3hPa | 1008.9hPa | 1007.1hPa | 1009hPa | 1010.6hPa | 1011.5hPa | 1012.1hPa |\n| 湿度 | 90.3% | 77.3% | 73.1% | 70.4% | 63.8% | 96.8% | 94.8% | 91.9% |\n| 云量 | 70% | 70% | 70% | 76.7% | 70% | 80% | 80% | 80% |', 'score': 0.77446437}, {'title': '今天白天北京晴转阴傍晚有雷阵雨需防范强对流

In [3]:
result = agent_executor.invoke("上海呢？")
# 处理响应数据
print(result)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Search
Action Input: 上海今天的天气情况
```
Observation: [{'title': '中国气象局-天气预报-城市预报- 上海', 'url': 'https://weather.cma.cn/web/weather/58367.html', 'content': '时间 23:00 02:00 05:00 08:00 11:00 14:00 17:00 20:00\n天气Image 16Image 17Image 18Image 19Image 20Image 21Image 22Image 23\n气温 31.1℃30.8℃30.6℃30.8℃31.1℃30.4℃29.4℃27.6℃\n降水 0.2mm 无降水 无降水 0.5mm 1.2mm 1.8mm 7.3mm 1.2mm\n风速 3.2m/s 3.3m/s 3m/s 3.3m/s 3.3m/s 3.3m/s 2.9m/s 2.7m/s\n风向 东北风 东北风 西北风 西北风 东北风 西北风 西北风 西北风\n气压 1008.9hPa 1007.9hPa 1007.9hPa 1009.5hPa 1009.4hPa 1008.4hPa 1009.3hPa 1010.1hPa\n湿度 81.6%83.8%87.8%84.8%82.4%86%87%79.4%\n云量 99.5%98.7%94.7%95.3%88.1%98.4%94.3%91.7% [...] 时间 08:00 11:00 14:00 17:00 20:00 23:00 02:00 05:00\n天气Image 24Image 25Image 26Image 27Image 28Image 29Image 30Image 31\n气温 30.8℃31.1℃30.4℃29.4℃27.6℃25.9℃25.7℃25.8℃\n降水 0.5mm 1.2mm 1.8mm 7.3mm 1.2mm 0.6mm 0.6mm 0.6mm\n风速 3.3m/s 3.3m/s 3.3m/s 2.9m/s 2.7m/s 3.1m/s 3.1m/s 3m/s\n风向 